# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Anderson Jiménez Torres <br>
Url: https://github.com/97anderson/AlgoritmosDeOptimizacion.git<br>
Google Colab: https://drive.google.com/file/d/1QktyyzJeQ4sk8noL5wNOFibgXHuQ0tXR/view?usp=drive_link  <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las 
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de 
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de 
grabación independientemente del número de tomas que se graben. No es posible grabar más 
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los 
servicios de los actores de doblaje sea el menor posible. Los datos son:
Número de actores: 10
Número de tomas : 30
Actores/Tomas : https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario








                                        

# Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

#### ¿Como represento el espacio de soluciones?
El espacio de soluciones se representa mediante la asignación de actores a escenas y de escenas o tomas a dias. Esto se realiza mediante la matriz 
escenas_x_actor, donde cada fila representa una toma y cada columna representa un actor. Un valor de 1 en una posición indica que el actor 
participa en esa toma, mientras que un valor de 0 indica lo contrario.


#### ¿Cual es la función objetivo?
Maximizar la cantidad de tomas que se pueden realiza con tal de reducir costos, creando un horario o planeación que no haga que un actor vaya mas de
lo necesario y de esa manera pagando lo menos posible. Este problema busca minimizar el gasto total por los servicios de los actores de doblaje. Esto implica encontrar la asignación óptima de las tomas a los días de grabación de manera que se cumplan todas las restricciones (como el límite de 6 tomas por día) y se minimice el costo total.

Matematicamente podemos denotar:
- Ci como el costo asociado al día de grabación i.
- ni como el número de tomas programadas para el día i.
- T, número total de dias de grabación
- G, como el gasto total por el servicio de los actores de doblaje

Es decir, la función objetivo es la suma de los costos asociados a cada día de grabación.

El desafío del algoritmo es encontrar la asignación de tomas a los días de grabación que minimice esta función objetivo, teniendo en cuenta las restricciones, como el límite de 6 tomas por día y la disponibilidad de los actores para cada t.te.


Para la solución del problema anterior se realizan dos desarrollos con dos tipos de algoritmos diferentes, pero que al final dan la misma configuración de días, el primero realizado es un **algoritmo de optimización de tipo Greedy (voraz)** y el segundo es un **algoritmo de búsqueda local.** La búsqueda local es una metaheurística que parte de una solución inicial y busca mejorarla iterativamente explorando el espacio de soluciones vecinas.

## ALGORITMO DE OPTIMIZACIÓN DE TIPO GREEDY (VORAZ):

En el algoritmo desarrollado se realiza una busqueda por actores que menos participan en el doble y se crea una matriz de menos a mas, se acomoda los 
actores que menos participan a los que mas participan. Esto con el fin de no hacer venir mas días de los necesarios al mismo actor, con tal de reducir costos en el doblaje.<br>mente.

Algunas caracteristicas son: 

- Objetivo de minimización de costos: El objetivo del algoritmo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Esto indica que el algoritmo está buscando una solución que minimice los costos.
- Restricciones específicas: Se establece una restricción clara de que no es posible grabar más de 6 tomas por día.
- Heurística de selección de actores y tomas: El algoritmo selecciona los actores y las tomas de manera que se satisfagan las restricciones y se busque minimizar los costos. No parece haber una búsqueda exhaustiva de todas las combinaciones posibles, sino que se toman decisiones "voraces" en cada paso basadas en la información disponible hasta el momento.

A continuación se comienza a desarrollar el código de lo que se explico anteriormente.

In [1]:
import pandas as pd
import numpy as np
import random
import math

In [3]:
# Se crea dataset a partir de archivo csv suministrado
datos_doblaje = pd.read_csv('Datos_problema_doblaje.csv', delimiter=',')
print(datos_doblaje)

   Unnamed: 0  Actor  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0        Toma    1.0         2.0         3.0         4.0         5.0   
1           1    1.0         1.0         1.0         1.0         1.0   
2           2    0.0         0.0         1.0         1.0         1.0   
3           3    0.0         1.0         0.0         0.0         1.0   
4           4    1.0         1.0         0.0         0.0         0.0   
5           5    0.0         1.0         0.0         1.0         0.0   
6           6    1.0         1.0         0.0         1.0         1.0   
7           7    1.0         1.0         0.0         1.0         1.0   
8           8    1.0         1.0         0.0         0.0         0.0   
9           9    1.0         1.0         0.0         1.0         0.0   
10         10    1.0         1.0         0.0         0.0         0.0   
11         11    1.0         1.0         1.0         0.0         1.0   
12         12    1.0         1.0         1.0         1.0        

#### ¿Como implemento las restricciones?
Las restricciones como el enunciado inicial es que no se realicen mas de 6 tomas por día y reducir las ida de cada actor al minimo posible con tal de que el actor vaya solo lo necesario, si en el mismo dia que va se pueden hacer todas las tomas en las que va a participar mucho mejor.<br>

Por ende se realiza un recorrido de los actores del que menos participa al que mas y se van revisando las escenas en las que participa, se descuenta la escena del maximo de tomas que hace el actor indicado por la ultima fila del archivo csv, teniendo en cuenta o llevando un contador de las 6 tomas permitidas en el día, el código es el siguiente: 

In [4]:
# DECLARACION DE VARIABLES GLOBALES
cantidad_escenas = datos_doblaje.iloc[1:-2, 0].copy().values.astype(int)  # numero total de escenas, sacadas del archivo csv
num_actores = len(datos_doblaje.columns) - 3  # Cantidad total de actores de la pelicula, sacando columnas que no sirven
contadores_por_actor = [0] * num_actores  # Se inicializa array con tamaño total de actores para sumar las escenas diarias, no > 6
escenas_x_actor = datos_doblaje.iloc[1:-2, 1:-2].copy().values.astype(int)  # Matriz con escenas en que participa cada actor
dias_grabacion = 1  # Se inicializa dia de grabacion en 1
sesiones_por_dia = {}  # diccionario que tendra las escenas por dias de grabación
total_actores_escena = datos_doblaje.iloc[1:-2, -1].copy().values.astype(int)  # Total de actores por escenas 

total_escenas_actores = datos_doblaje.iloc[[0, -1], 1:num_actores + 1] # Se consigue cantidad de escenas de cada actor
actores_con_escenas = total_escenas_actores.values.astype(int)
array_total_tomas_actor = actores_con_escenas[1]
indice_orden = np.argsort(actores_con_escenas[1])  # Orden de menor a mayor de fila final
actores_con_escenas = actores_con_escenas[:, indice_orden]  # Aplicamos el mismo orden a la primera fila

contador_escenas = len(cantidad_escenas)
tomas_totales = []
horario_doblaje = {}
tomas_programadas = {}  # Cambiamos la lista a un diccionario para poder agregar elementos con claves
contador_tomas = 0

# Se valida actor con escenas minimas a empezar a validar tomando variable actores_con_escenas fila 2
for posicion in range(num_actores):
    actor = actores_con_escenas[0, posicion]  # Se valida actor que menos participa a max
    numero_escenas = 0
    contadores_por_actor = [0] * num_actores
    actores_participan_tomas = 0  # variable que identifica los actores que participan en una toma que se va a realizar.
    
    while numero_escenas < len(cantidad_escenas):
        escenas_asignadas_dia = []  # Lista para almacenar las escenas asignadas en el día actual
        
        for escena in range(len(cantidad_escenas)):
            numero_escenas += 1
            if (escenas_x_actor[escena, actor - 1] == 1 and 
                contador_escenas > 0 and 
                escena + 1 not in tomas_totales):
                
                participantes = [i for i in range(num_actores) if escenas_x_actor[escena, i] == 1]
                
                for i in participantes:
                    contadores_por_actor[i] += 1
                    if contadores_por_actor[i] < 6:
                        actores_participan_tomas += 1
                    
                if (contador_tomas < 6 and 
                    actores_participan_tomas >= total_actores_escena[escena]):
                    
                    sesiones_por_dia.setdefault(dias_grabacion, []).append(escena)
                    contador_escenas -= 1
                    tomas_programadas.setdefault(dias_grabacion, []).append(escena)
                    escenas_asignadas_dia.append(escena)  # Añadir la escena a las asignadas en el día actual
                    contador_tomas += 1
                    tomas_totales.append(escena + 1)
                    cantidad_escenas -= 1

                    for i in participantes:
                        array_total_tomas_actor[i] -= 1
                
                actores_participan_tomas = 0
                
                if contador_tomas >= 6:
                    dias_grabacion += 1
                    contadores_por_actor = [0] * num_actores
                    numero_escenas = 0
                    contador_tomas = 0
                    break  # Salir del bucle de escenas por actor para pasar al siguiente día


Finalmente después de hacer el recorrido anterior e ir revisando actor y tomas o escenas de cada actor y agregarlas a un dia determinado, hemos hallado una configuración optima para el horario en el que podemos hacer ir a los actores al doblaje de la pelicula.
El resultado optenido es el siguiente:

In [6]:
# Imprimir el horario de doblaje por día
for dia, sesiones in sesiones_por_dia.items():
    if sesiones:
        print(f"Día {dia}: {len(sesiones)} sesiones")
        for sesion in sesiones:
            participantes = [i + 1 for i, valor in enumerate(escenas_x_actor[sesion]) if valor == 1]
            print(f"    Escena {sesion + 1}: Participantes {participantes}")
    else:
        print(f"Día {dia}: 0 sesiones")

Día 1: 6 sesiones
    Escena 10: Participantes [1, 2, 6, 9]
    Escena 26: Participantes [1, 3, 5, 9]
    Escena 16: Participantes [4, 10]
    Escena 25: Participantes [1, 2, 4, 10]
    Escena 3: Participantes [2, 5, 7]
    Escena 4: Participantes [1, 2, 7, 8]
Día 2: 6 sesiones
    Escena 15: Participantes [1, 2, 7]
    Escena 5: Participantes [2, 4, 8]
    Escena 11: Participantes [1, 2, 3, 5, 8]
    Escena 21: Participantes [6, 8]
    Escena 8: Participantes [1, 2, 6]
    Escena 12: Participantes [1, 2, 3, 4, 6]
Día 3: 6 sesiones
    Escena 14: Participantes [1, 3, 6]
    Escena 18: Participantes [3, 6]
    Escena 24: Participantes [3, 6]
    Escena 29: Participantes [1, 5, 6]
    Escena 1: Participantes [1, 2, 3, 4, 5]
    Escena 2: Participantes [3, 4, 5]
Día 4: 6 sesiones
    Escena 6: Participantes [1, 2, 4, 5]
    Escena 7: Participantes [1, 2, 4, 5]
    Escena 13: Participantes [1, 4, 5]
    Escena 20: Participantes [1, 3, 4, 5]
    Escena 27: Participantes [4, 5]
    Escena 17

# Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

El problema anterior resulto por el código descrito tiene una complejidad que depende de varios factores, entre ellos el número de tomas o escenas, 
el número de actores y la distribución de la participación de los actores en las tomas.

#### Orden depl coejidad
- El bucle principal itera sobre todos los actores y luego sobre todas las escenas, lo que da como resultado una complejidad de tiempo de O(num_actores * num_escenas).
- Dentro de este bucle, hay operaciones adicionales como verificar la disponibilidad de los actores para una escena específica, lo que agrega cierta complejidad adicional.
- La asignación de escenas a actores se realiza de manera voraz, lo que puede llevar a una solución subóptima pero rápida. Por lo tanto, la complejidad real puede variar según la distribución de las escenas y la participación de los actores.dad

#### Contabilizar el espacio de soluciones
- El espacio de soluciones puede contabilizarse examinando todas las combinaciones posibles de asignación de actores a escenas.
- Dado que para cada escena hay un número finito de actores posibles y el número de escenas es finito, el espacio de soluciones será la cantidad total de combinaciones mosibles de asignaciones de actores a escenas.
- Esta cantidad puede ser enorme, especialmente en películas con muchas escenas y actores. Por lo tanto, el enfoque voraz utilizado en el código puede ser beneficioso para reducir el tiempo de búsqueda de una solución factible, aunque puede no ser óptimo en términos de encontrar la mejor solución posible.

# Diseño
- ¿Que técnica utilizo? ¿Por qué?

El código proporcionado utiliza una técnica de programación voraz (greedy) para resolver el problema de asignar escenas de doblaje a actores. 
Se elige esta técnica por varias razones:

- Eficiencia en tiempo de ejecución: Los algoritmos voraces suelen ser más rápidos que otros enfoques, ya que toman decisiones locales en cada paso sin tener en cuenta el futuro. Esto los hace adecuados para problemas donde se necesita una solución rápida.
- Facilidad de implementación: La implementación de algoritmos voraces tiende a ser más simple y directa en comparación con otros enfoques como la programación dinámica o los algoritmos genéticos.
- Adecuación al problema: En este caso específico, el problema de asignar escenas de doblaje a actores puede abordarse de manera voraz, ya que en cada paso se selecciona la próxima escena disponible y se asignan actores a ella hasta que se cumplan todas las restricciones. No hay una necesidad inmediata de considerar todas las posibles combinaciones de asignaciones de actores a escenas, lo que hace que un enfoque voraz sea adecuado.

## ALGORITMO DE BÚSQUEDA LOCAL:

El algoritmo implementado es un algoritmo de búsqueda local. La búsqueda local es una metaheurística que parte de una solución inicial y busca mejorarla iterativamente explorando el espacio de soluciones vecinas. En este caso, la solución se representa como un diccionario donde las claves son los días de grabación y los valores son las sesiones asignadas a cada día.<br>

El desarrollo del algoritmo aplicando tecnicas Metaheuristica (Búsqueda local):

In [8]:
# Metodo para calcular el costo total de una solución
def calculate_cost(solucion, datos_doblaje):
    costo_total = 0
    for dia, sesiones in solucion.items():
        for sesion in sesiones:
            costo_total += datos_doblaje.iloc[sesion, -2]
    return costo_total

# metodo para generar una solución vecina intercambiando dos sesiones entre días
def generate_neighbor_solution(solucion_actual):
    vecino = solucion_actual.copy()
    dias = list(vecino.keys())
    np.random.shuffle(dias)  # Días aleatoriamente
    dia_1, dia_2 = dias[:2]  # Días aleatorios

    if vecino[dia_1] and vecino[dia_2]:
        # Intercambiar dos sesiones aleatorias entre los dos días seleccionados
        sesion_1 = np.random.choice(vecino[dia_1])
        sesion_2 = np.random.choice(vecino[dia_2])
        vecino[dia_1].remove(sesion_1)
        vecino[dia_2].remove(sesion_2)
        vecino[dia_1].append(sesion_2)
        vecino[dia_2].append(sesion_1)

    return vecino

# Metodo para ejecutar el algoritmo de búsqueda local
def local_search(datos_doblaje):
    solucion_actual = {}
    tomas_restantes = set(range(len(datos_doblaje) - 2))
    tomas_restantes.remove(0)
    costo_actual = 0
    
    # Inicializar la solución con sesiones aleatorias por día
    for dia in range(1, len(datos_doblaje) - 2):
        solucion_actual[dia] = []
        while len(solucion_actual[dia]) < 6 and tomas_restantes:
            sesion = np.random.choice(list(tomas_restantes))
            solucion_actual[dia].append(sesion)
            tomas_restantes.remove(sesion)
            costo_actual += int(datos_doblaje.iloc[sesion, -1])

    # Mejorar la solución iterativamente
    while True:
        vecino = generate_neighbor_solution(solucion_actual)
        costo_vecino = calculate_cost(vecino, datos_doblaje)
        if costo_vecino < costo_actual:
            solucion_actual = vecino
            costo_actual = costo_vecino
        else:
            break

    return solucion_actual, costo_actual

# Ejecutar algortimo llamando a busqueda local, retorna la solución, asi como también el costo optimo
solucion_optima, costo_optimo = local_search(datos_doblaje)
dias_con_tomas = {dia: sesiones for dia, sesiones in solucion_optima.items() if sesiones}

for dia, sesiones in dias_con_tomas.items():
    if sesiones:
        print(f"Día {dia}: {len(sesiones)} sesiones")
        for sesion in sesiones:
            if sesion < len(escenas_x_actor):
                participantes = [i + 1 for i, valor in enumerate(escenas_x_actor[sesion]) if valor == 1]
                print(f"    Escena {sesion + 1}: Participantes {participantes}")
    else:
        print(f"Día {dia}: 0 sesiones")
print(f"Costo total: {costo_optimo}")

Día 1: 6 sesiones
    Escena 16: Participantes [4, 10]
    Escena 8: Participantes [1, 2, 6]
    Escena 9: Participantes [1, 2, 4]
    Escena 11: Participantes [1, 2, 3, 5, 8]
    Escena 7: Participantes [1, 2, 4, 5]
    Escena 30: Participantes [1, 4]
Día 2: 6 sesiones
    Escena 2: Participantes [3, 4, 5]
    Escena 14: Participantes [1, 3, 6]
    Escena 29: Participantes [1, 5, 6]
    Escena 12: Participantes [1, 2, 3, 4, 6]
    Escena 20: Participantes [1, 3, 4, 5]
    Escena 6: Participantes [1, 2, 4, 5]
Día 3: 6 sesiones
    Escena 23: Participantes [1, 3]
    Escena 17: Participantes [1, 3]
    Escena 3: Participantes [2, 5, 7]
    Escena 10: Participantes [1, 2, 6, 9]
    Escena 22: Participantes [1, 2, 3, 4]
    Escena 27: Participantes [4, 5]
Día 4: 6 sesiones
    Escena 5: Participantes [2, 4, 8]
    Escena 21: Participantes [6, 8]
    Escena 25: Participantes [1, 2, 4, 10]
    Escena 4: Participantes [1, 2, 7, 8]
    Escena 28: Participantes [1, 4]
    Escena 13: Participan

# Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

La complejidad y funciones del algoritmo anterior es:

- **Calcular costo:** Esta función calcula el costo de una solución. realiza un recorrido de todas las grabaciones en cada dia y realiza una suma de los costos asociados a las tomas asignadas para el día.
- **Solucion vecina:** Esta función genera una solución vecina intercambiando dos sesiones entre dos días de grabación aleatorios. Esto se hace para explorar nuevas soluciones en el espacio de búsqueda.
- **Busqueda local:** Esta función implementa el algoritmo de búsqueda local. Comienza con una solución inicial aleatoria y luego mejora iterativamente la solución buscando soluciones vecinas que mejoren el costo total. El proceso continúa hasta que no se pueda encontrar una solución vecina mejor.meSe ejecuta la búsqueda local utilizando la función busqueda_local. La solución óptima y su costo óptimo se almacenan y se imprime la información sobre las sesiones asignadas a cada día de grabación, así como el costo total de la solución óptima.

# Diseño
- ¿Que técnica utilizo? ¿Por qué?

Se utiliza la búsqueda local. La búsqueda local es una técnica heurística que parte de una solución inicial y realiza pequeños cambios en esta solución para mejorarla iterativamente. La razón principal para utilizar la búsqueda local en este contexto es su capacidad para explorar el espacio de soluciones de manera eficiente y encontrar soluciones de alta calidad en problemas combinatorios como el de asignación de sesiones de doblaje a días de grabación.

Algunas de las razones para aplicar este algoritmo son:
1. **Adaptabilidad:** La búsqueda local puede adaptarse fácilmente a diferentes tipos de problemas y restricciones, lo que la hace versátil y aplicable a una amplia gama de situaciones.
2. **Eficiencia computacional:** La búsqueda local es computacionalmente eficiente en comparación con otros métodos de optimización, lo que la hace adecuada para problemas con un gran espacio de búsqueda como este.
3. **Facilidad de implementacion:** La lógica de la búsqueda local es relativamente simple de implementar en comparación con otras técnicas más complejas, lo que permite una implementación rápida y efectiva.
4. **Exploración espacio de soluciones:** La búsqueda local es capaz de explorar de manera efectiva el espacio de soluciones vecinas, lo que puede conducir a soluciones de alta calidad incluso en problemas con múltiples óptimos locales.